In [1]:
import pandas as pd
from database.market import Market
from database.strategy import Strategy
from database.sec import SEC
from modeler.modeler import Modeler as m
from datetime import datetime, timedelta, timezone
import numpy as np
import math
from tqdm import tqdm
import pickle
from sklearn.preprocessing import OneHotEncoder

In [2]:
sec = SEC()
market = Market()

TypeError: Strategy.__init__() missing 1 required positional argument: 'name'

In [ ]:
market.connect()
sp5 = market.retrieve("sp500")
prices = market.retrieve("prices")
market.disconnect()

In [5]:
prices["date"] = pd.to_datetime(prices["date"])
prices["year"] = [x.year for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
for points in [100,200,500]:
    print(points,len(prices[prices["adjClose"]>=points]["ticker"].unique()))
    quarterly_grouped = prices.groupby(["year","quarter","ticker"]).mean().reset_index()
categories = []
for value in quarterly_grouped["adjClose"]:
    if value > 0 and value <= 100:
        categories.append(100)
    else:
        if value > 100 and value <= 200:
            categories.append(200)
        else:
            categories.append(500)
quarterly_grouped["category"] = categories
quarterly_grouped.reset_index(inplace=True)
groups = quarterly_grouped.merge(sp5.rename(columns={"Symbol":"ticker"}),on="ticker",how="left")
g = groups[["year","quarter","ticker","adjClose","category","GICS Sector","CIK"]]
g.dropna(inplace=True)
g["string_category"]  = [str(x) for x in g["category"]]
g["classification"] = g["string_category"] + g["GICS Sector"]
numberss = len(g["classification"].unique())
enc = OneHotEncoder(handle_unknown="ignore")
transformed = [[x] for x in g["classification"]]
encoding = enc.fit_transform(transformed)
df_encoding = pd.DataFrame(encoding.toarray())
for col in df_encoding.columns:
    g[col] = df_encoding[col]

In [12]:

market.connect()
market.store("unified_financials",data)
market.disconnect()

 89%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏             | 440/493 [04:57<00:25,  2.11it/s]

prep FRC 'filed'


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2240/2240 [05:05<00:00,  7.33it/s]


In [73]:
year_range = range(2021,2022)
yearly_gap = 1
for training_years in range(4,5):
    for year in tqdm(year_range):
        try:
            training_data = data[(data["year"] <= year) & (data["year"] >= year - training_years - 1)]
            factors = list(data.columns)
            factors = [x for x in factors if x not in ["year","quarter","ticker","filed"]]
            for i in range(numberss):
                try:
                    factors.remove(str(i))
                except:
                    continue
            for col in factors:
                training_data[col].replace([np.inf,-np.inf,np.nan,np.NaN],training_data[col].mean(),inplace=True)
            training_data.dropna(inplace=True)
            ## resetting the factor reporting dates
            temp = [x for x in factors]
            temp.extend(["year","quarter","ticker"])
            x_plz = training_data[temp]
            x_plz["year"] = x_plz["year"] + yearly_gap
            temp = [str(x) for x in range(numberss)]
            temp.extend(["year","quarter","ticker"])
            y_plz = training_data[temp]
            training_data = x_plz.merge(y_plz,on=["year","quarter","ticker"],how="left")
            ## remerging
            relevant = training_data[(data["year"] < year) & (data["year"] >= year - training_years - 1)]
            relevant.dropna(inplace=True)
            x = relevant[factors]
            y = relevant[[str(x) for x in range(numberss)]]
            prediction_data = training_data[training_data["year"]==year]
            refined_data = {"X":x.reset_index(drop=True),"y":y.reset_index(drop=True)}
            models = m.xgb_classify(refined_data.copy(),multioutput=True)
            model = models["model"]
            for col in factors:
                prediction_data[col].replace([np.inf,-np.inf,np.nan,np.NaN],prediction_data[col].mean(),inplace=True)
            predictions = enc.inverse_transform(model.predict(prediction_data[factors]))
            prediction_data["prediction"] = [x[0] for x in predictions]
            prediction_data["score"] = models["score"].item()
            sim = prediction_data[["year","quarter","ticker","prediction","score"]]
            sim["training_years"] = training_years
            strat.connect()
            strat.store("predicted_stock_categories",sim)
            strat.disconnect()
        except Exception as e:
            print(year,str(e))

  0%|                                                                                                                                             | 0/4 [00:00<?, ?it/s]C:\Users\ericj\AppData\Roaming\Python\Python39\site-packages\pandas\core\generic.py:6610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\Users\ericj\AppData\Roaming\Python\Python39\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\ericj\AppData\Local\Temp/ipykernel_11956/2179828245.py:21: SettingWithCopyWarning: 
A value is trying to be s

C:\Users\ericj\AppData\Roaming\Python\Python39\site-packages\pandas\core\generic.py:6610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\Users\ericj\AppData\Local\Temp/ipykernel_11956/2179828245.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction_data["prediction"] = [x[0] for x in predictions]
C:\Users\ericj\AppData\Local\Temp/ipykernel_11956/2179828245.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

 25%|█████████████████████████████████▎                                                                                                   | 1/4 [00:02<00:06,  2.19s/it]C:\Users\ericj\AppData\Roaming\Python\Python39\site-packages\pandas\core\generic.py:6610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\Users\ericj\AppData\Roaming\Python\Python39\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\ericj\AppData\Local\Temp/ipykernel_11956/2179828245.py:21: SettingWithCopyWarning: 
A value is trying to be s

C:\Users\ericj\AppData\Roaming\Python\Python39\site-packages\pandas\core\generic.py:6610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\Users\ericj\AppData\Local\Temp/ipykernel_11956/2179828245.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction_data["prediction"] = [x[0] for x in predictions]
C:\Users\ericj\AppData\Local\Temp/ipykernel_11956/2179828245.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

 50%|██████████████████████████████████████████████████████████████████▌                                                                  | 2/4 [00:06<00:06,  3.21s/it]C:\Users\ericj\AppData\Roaming\Python\Python39\site-packages\pandas\core\generic.py:6610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\Users\ericj\AppData\Roaming\Python\Python39\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\ericj\AppData\Local\Temp/ipykernel_11956/2179828245.py:21: SettingWithCopyWarning: 
A value is trying to be s

C:\Users\ericj\AppData\Roaming\Python\Python39\site-packages\pandas\core\generic.py:6610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\Users\ericj\AppData\Local\Temp/ipykernel_11956/2179828245.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction_data["prediction"] = [x[0] for x in predictions]
C:\Users\ericj\AppData\Local\Temp/ipykernel_11956/2179828245.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

 75%|███████████████████████████████████████████████████████████████████████████████████████████████████▊                                 | 3/4 [00:13<00:04,  4.56s/it]C:\Users\ericj\AppData\Roaming\Python\Python39\site-packages\pandas\core\generic.py:6610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\Users\ericj\AppData\Roaming\Python\Python39\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\ericj\AppData\Local\Temp/ipykernel_11956/2179828245.py:21: SettingWithCopyWarning: 
A value is trying to be s

In [74]:
sim

,year,quarter,ticker,prediction,score,training_years
16,2021,1,AAPL,None,0.053279,4
17,2021,2,AAPL,None,0.053279,4
18,2021,3,AAPL,None,0.053279,4
19,2021,4,AAPL,None,0.053279,4
38,2021,1,ABMD,500Consumer Discretionary,0.053279,4
...,...,...,...,...,...,...
11148,2021,4,CARR,None,0.053279,4
11151,2021,2,OTIS,100Energy,0.053279,4
11152,2021,3,OTIS,100Energy,0.053279,4
11153,2021,4,OTIS,None,0.053279,4


In [13]:
base = g[["year","quarter","ticker","classification"]]
base.rename(columns={"classification":"prediction"},inplace=True,errors="ignore")
base["score"] = 1
base["training_years"] = 0
base

,year,quarter,ticker,prediction,score,training_years
0,1994,1,AAPL,100Information Technology,1,0
1,1994,1,ABMD,100Health Care,1,0
2,1994,1,ABT,100Health Care,1,0
3,1994,1,ADBE,100Information Technology,1,0
4,1994,1,ADI,100Information Technology,1,0
...,...,...,...,...,...,...
46927,2021,4,XEL,100Utilities,1,0
46928,2021,4,XLNX,200Information Technology,1,0
46929,2021,4,XOM,100Energy,1,0
46930,2021,4,XRAY,100Health Care,1,0


In [14]:
strat.connect()
strat.store("predicted_stock_categories",base)
strat.disconnect()